 ## Puzle de 8 con RL
 
Se aplica el algoritmo Q-learning para diseñar un agente que aprenda a resolver el puzzle del 8. 
 

Para poder aplicar Q-learning necesitamos numerar las diferentes acciones y estados para poder crear la q_table.

In [2]:
import numpy as np
import random
import math
from IPython.display import clear_output

In [3]:
from search import Problem
import itertools
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
        
    
    def actions(self,estado):
     
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (0,3,6): 
            accs.append("Mover hueco izquierda") 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0
    
    def lista_estados(self,estado):
        i = 0;
        diccionario = {}
        for e in itertools.permutations(estado):
            diccionario[e] = i
            i = i + 1;
        return diccionario  
    
    def listaAcciones(self):
        listaAcciones = {}
        listaAcciones[0] ="Mover hueco arriba" 
        listaAcciones[1] ="Mover hueco abajo"
        listaAcciones[2] ="Mover hueco derecha"
        listaAcciones[3] ="Mover hueco izquierda"
        return listaAcciones
    
    def listaAcciones2(self):
        listaAcciones2 = {}
        listaAcciones2["Mover hueco arriba"] = 0
        listaAcciones2["Mover hueco abajo"] =1
        listaAcciones2["Mover hueco derecha"] =2
        listaAcciones2["Mover hueco izquierda"] =3
        return listaAcciones2

In [4]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])    

In [5]:
initial=(1,2,0,4,5,3,7,8,6)

In [6]:
ocho = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho.__init__(initial) 
state = ocho.initial


estado objetivo (1, 2, 3, 4, 5, 6, 7, 8, 0)


In [7]:
def recompensa(state):
    if state==ocho.goal:
        return 100
    else:
        return 0

In [8]:
def agente_puzle8(episodios=10000, alpha = 0.7, gamma = 0.6, epsilon = 0.8):
    """Creamos y entrenamos el agente con los parametros dados"""
    state = ocho.initial
    listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    exitos=0
    for i in range(1, episodios):
        if random.uniform(0, 1) < epsilon:
            mejor = listaAcciones2[random.choice(ocho.actions(state))]
            #print("La accion aleatoria seleccionada es ", listaAcciones[mejor])
        else:
            mejor = 0
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor]:
                    mejor = accion
                    #print("La mejor accion es ", listaAcciones[mejor])

        if (listaAcciones[mejor] in ocho.actions(state)):
            sig_estado = ocho.result(state , listaAcciones[mejor])
            reward = recompensa(sig_estado)
        else:
            reward = -50 #accion no aplicable
            sig_estado =state
        old_value = q_table[listaEstados[state]][mejor]
        next_max = np.max(q_table[listaEstados[sig_estado]][mejor])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[listaEstados[state], mejor] = new_value
        state = sig_estado
        if state == ocho.goal:
            #Termino el episodio con éxito. Se ha encontrado la solución
            state = ocho.initial
            exitos=exitos+1
            continue
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n") 
    print("La Q-table:\n") 
    print(q_table)
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos: ", exitos)

In [9]:
# MODIFICA LOS PARÁMETROS Y OBSERVA EL APRENDIZAJE
alpha = 0.5
gamma = 0.9
epsilon = 0.8
episodios=100000

In [10]:
# puedes medir el tiempo de aprendizaje 
#%%time
agente_puzle8(episodios,alpha,gamma,epsilon)

Episode: 99900
Training finished.

La Q-table:

[[-25.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 ...
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0.   0.   0.   0.]]
Número de éxitos:  1


In [11]:
def evalua_agente_puzle8(initial):
    """Solo explotación del agente sin modificar la tabla Q"""
    exitos=0
    fallos=0
    state = initial
    episodios =100
    #listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    for i in range(1, episodios+1):
        done =False
        while not done:
            mejor_accion = 0        
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor_accion]:
                    mejor_accion = accion                
            print("La accion aleatoria seleccionada es ", listaAcciones[mejor_accion])
            print("Estado:", state)
            if (listaAcciones[mejor_accion] in ocho.actions(state)):
                state = ocho.result(state , listaAcciones[mejor_accion])
                print("Estado siguiente:", state)
            else: 
                #termino con fallo porque la accion no es aplicable
                fallos=fallos+1
                done = True            
            if state == ocho.goal:
                #print("Termino el episodio con éxito. Se ha encontrado la solución")
                exitos=exitos+1 
                done =True 
                state=initial
            if i % 100 == 0:
                clear_output(wait=True)
                print(f"Episodio: {i}")
        
    print("Evaluación terminada.\n") 
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos:", exitos)
    print("Número de éxitos:", (exitos/episodios)*100, " %")  
    

In [12]:
evalua_agente_puzle8(initial)

Episodio: 100
Evaluación terminada.

Número de éxitos: 100
Número de éxitos: 100.0  %


¿Funciona bien el entrenamiento?
¿Puedes mejorarlo?

In [12]:
initial2=(1,5,7,4,2,3,0,8,6)
ocho2 = Ocho_Puzzle(initial2)
print("estado objetivo",ocho.goal)   
ocho2.__init__(initial) 
state = ocho2.initial

estado objetivo (1, 2, 3, 4, 5, 6, 7, 8, 0)


El agente ha aprendido con QLearning a resolver el problema del 8 puzle empezando en un estado inicial fijo y ha tardado un cierto tiempo.
Una vez entrenado puedes ver qué tal se comporta (usando sólo la parte de explotación de la matriz Q aprendida y midiendo el porcentaje de éxitos). 

#### Responde a las siguientes cuestiones:

- Justifica de forma razonada si esta aproximación con aprendizaje es mejor que la aproximación con búsqueda heurística de la primera práctica.
- Observa la matriz de aprendizaje ¿qué ha aprendido el agente?
- Si entrenas el agente para un estado inicial ¿cómo se comporta al evaluarlo con otro estado inicial? 
- Realiza los cambios necesarios para que el agente aprenda a resolver el problema desde cualquier estado inicial.
- Te parece adecuada la función de recompensa utilizada. ¿Crees que se podría mejorar el rendimiento del agente cambiando la recompensa?
- Comprueba cómo se comporta el aprendizaje con distintas configuraciones de los parámetros. Indica claramente cuál es la mejor configuración que has encontrado.

1. Es mejor la forma con RL debido a que, a pesar de que al principio pueda ser más costoso, el resto de veces que lo ejecutas (tras entrenarlo) el coste es mucho mejor debido a que ya sabe cuales son las mejores opciones en cada estado. Sin embargo, aunque la versión heurística pueda parecer menos costosa en la primera ejecución, no hay ninguna mejora en el coste debido a que no aprende nada. Por lo tanto, a la larga es mejor RL.

2. Observando la matriz de aprendizaje, vemos que hay varias casillas que tienen valores negativos. Esto quiere decir que escogiendo esa opción no vamos a llegar a una solución válida sin pasar por movimientos "ilegales". Sin embargo, aunque en las filas mostradas de la Q-tabla al hacer print no veamos valores cercanos al 100 (debido a que esta "porción" de tabla no es la tabla entera) sabemos que en el resto de la tabla tiene que haber posiciones que tengan necesariamente esos valores, debido a que sí que hay estados prometedores. No siempre vemos -50 o 100 debido a los parámetros seleccionados (alpha, gamma y epsilon), sino que se van "relajando" por la fórmula aplicada.

3. Al evaluar el agente con el estado inicial2, habiendo sido entrenado con el estado inicial (inicial!=inicial2), observamos que en ninguna de las 100 ejecuciones tiene éxito. Esto se debe a que el agente ha sido entrenado con un único estado inicial en todos los episodios, lo cual lo "prepara" solamente para una cantidad de estados reducida. Por tanto, al cambiar el estado inicial para evaluarlo no sabe cómo resolver el problema al aparecer estados que nunca había visto antes. 

4.

In [13]:
#Hacemos un random de los estados iniciales para que entrene con varios estados iniciales y así la q-table esté rellena por lugares que probando con un solo estado esté vacía.
evalua_agente_puzle8(initial)

Episodio: 100
Evaluación terminada.

Número de éxitos: 100
Número de éxitos: 100.0  %


In [ ]:
for i in range (1, 1000):
    list1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
    initial = tuple(random.sample(list1, 10))
    ocho = Ocho_Puzzle(initial)
    ocho.__init__(initial)
    state = ocho.initial
    if(ocho.check_solvability(initial)):
        alpha = 0.5
        gamma = 0.9
        epsilon = 0.8
        episodios=100
        agente_puzle8(episodios,alpha,gamma,epsilon)

Para que funcione óptimamente desde cualquier estado inicial, lo que podemos hacer es llamar al entrenamiento desde varios estados iniciales (generados aleatoriamente) para que se rellene la q_table en todas las posiciones, ya que si solo probamos desde un único estado inicial, solo se rellena una parte reducida de la tabla, y si lo queremos aplicar a un estado inicial bastante diferente, es posible que no esté entrenado para él. Para que no de error, antes del llamar al agente entrenador, comprobamos que el estado aleatorio inicial que se ha creado sea resoluble.

5. Sí, consideramos que la recompensa utilizada es adecuada. Sin embargo, creemos que se podría mejorar añadiendo, a parte de la recompensa de 100 cuando se alcanza el estado final, una penalización por cada paso que se toma (para intentar llegar así en el menor número de pasos al puzle resuelto) y una recompensa por cada número que esté en su posición objetivo. De esta forma, aumentaremos la recompensa a medida que reducimos la "distancia" al estado objetivo.

6.

In [46]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])   
initial=(1,2,0,4,5,3,7,8,6)
ocho2 = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho2.__init__(initial) 
state = ocho2.initial
alpha = 0.1
gamma = 0.1
epsilon = 0.1
episodios=5
agente_puzle8(episodios,alpha,gamma,epsilon)
evalua_agente_puzle8(initial)

Episodio: 100
Evaluación terminada.

Número de éxitos: 100
Número de éxitos: 100.0  %


In [56]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])   
initial=(1,2,0,4,5,3,7,8,6)
ocho2 = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho2.__init__(initial) 
state = ocho2.initial
alpha = 0.9
gamma = 0.9
epsilon = 0.9
episodios=5
agente_puzle8(episodios,alpha,gamma,epsilon)
evalua_agente_puzle8(initial)

Episodio: 100
Evaluación terminada.

Número de éxitos: 0
Número de éxitos: 0.0  %


In [51]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])   
initial=(1,2,0,4,5,3,7,8,6)
ocho2 = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho2.__init__(initial) 
state = ocho2.initial
alpha = 0.5
gamma = 0.5
epsilon = 0.5
episodios=5
agente_puzle8(episodios,alpha,gamma,epsilon)
evalua_agente_puzle8(initial)

Episodio: 100
Evaluación terminada.

Número de éxitos: 100
Número de éxitos: 100.0  %


Hemos probado con bastantes combinaciones de parámetros (dejamos aquí reflejadas tres de nuestras pruebas) y hemos llegado a las siguientes conclusiones:

Hemos observado que con un número elevado de episodios de entrenamientos, siempre funciona mientras los parámetros sean estrictamente mayores que 0 o estrictamente menores que 1.
Ya una vez reducido el número de episodios de entrenamiento, podemos ver que poniendo todos los parámetros a 0.1 y vemos que funciona igualmente, mientras que poniéndolos todos a 0.9, deja de funcionar. Evidentemente, si funciona con los parámetros a 0.1, también va a funcionar con ellos inicializados a 0.5.
Además, hemos visto que lo óptimo es ir reduciendo dinámicamente todos los parámetros, pero para ello, necesitaríamos más episodios de entrenamiento, por ello, en este problema que es sencillo, no vamos a notar la diferencia.
